In [1]:
import sys
sys.path.append('/home/xilini/webnlg/submissions2020/corpus-reader-master/')

from benchmark_reader import Benchmark
from benchmark_reader import select_files
import json

# extract reference texts
b = Benchmark()
b.fill_benchmark([('/home/xilini/webnlg/submissions2020/testdata/rdf2text/en/',
                   'rdf-to-text-generation-test-data-with-refs-en.xml')])
b.b2json('./', 'data_REF.json')

In [2]:
with open('../inputs/sampled_ids.txt', 'r') as f2:
    sampled_ids = f2.read().splitlines()

In [3]:
with open('./data_REF.json', 'r') as f1:
    data_REF = json.load(f1)['entries']

In [6]:
sampled_data_REF = []
cats = {}
for item in data_REF:
    for key, value in item.items():
        if key in sampled_ids:
            sampled_data_REF.append(item)
            cats[key] = value['category']

In [13]:
cats

{'3': 'Airport',
 '29': 'Politician',
 '34': 'Artist',
 '68': 'MusicalWork',
 '80': 'Film',
 '83': 'MusicalWork',
 '128': 'WrittenWork',
 '129': 'Politician',
 '142': 'University',
 '152': 'MusicalWork',
 '165': 'Athlete',
 '169': 'Building',
 '184': 'Film',
 '187': 'MusicalWork',
 '188': 'MeanOfTransportation',
 '192': 'Film',
 '195': 'Building',
 '203': 'Artist',
 '207': 'Scientist',
 '224': 'Artist',
 '237': 'SportsTeam',
 '240': 'MusicalWork',
 '256': 'MusicalWork',
 '267': 'Company',
 '286': 'Film',
 '287': 'Film',
 '292': 'Artist',
 '300': 'University',
 '301': 'SportsTeam',
 '307': 'Film',
 '350': 'City',
 '377': 'Scientist',
 '383': 'MusicalWork',
 '388': 'Scientist',
 '390': 'Astronaut',
 '392': 'Monument',
 '417': 'MusicalWork',
 '419': 'Scientist',
 '441': 'MusicalWork',
 '448': 'MusicalWork',
 '452': 'MusicalWork',
 '455': 'Food',
 '483': 'Film',
 '489': 'Scientist',
 '496': 'Astronaut',
 '498': 'Athlete',
 '500': 'Company',
 '514': 'MusicalWork',
 '531': 'Artist',
 '533': 

In [51]:
lexicalisations = {}
import random
random.seed(0)

for entry in sampled_data_REF:
    lexs = []
    for k, v in entry.items():
        lex = v['lexicalisations']
        for i in lex:
            lexs.append(i['lex'])
    this_lex = random.sample(lexs, 1)
    lexicalisations[k] = this_lex[0]

In [55]:
with open('../inputs/texts_REF.json', 'w') as f3:
    json.dump(lexicalisations, f3)

In [14]:
# prepare triples
triplet_data = {}

# save info about triple size
triple_size = {}

for entry in sampled_data_REF:
    lexs = []
    rows = []
    for k, v in entry.items():
        n = 0
        triples = v['modifiedtripleset']       
        for t in triples:
            subj = t['subject'].replace('_', ' ')
            pred = t['property'].replace('_', ' ')
            obj = t['object'].replace('_', ' ')
            this_t = [subj, pred, obj]
            rows.append(this_t)
            n += 1
        triple_size[k] = n
                
    triplet_data[k] = []
    triplet_data[k].append(rows)


In [16]:
with open('./triple_size.json', 'w') as f3:
    json.dump(triple_size, f3)

In [61]:
html_triplet = '''

  <tr>
    <td class="tg-0lax">SUBJ</td>
    <td class="tg-0lax">PRED</td>
    <td class="tg-0lax">OBJ</td>
  </tr>

'''

In [62]:
html_head = '''

<div class="center">
<table class="tg">

<thead>
  <tr>
    <th class="tg-0pky"><b>Subject</b></th>
    <th class="tg-0pky"><b>Predicate</b></th>
    <th class="tg-0pky"><b>Object</b></th>
  </tr>
</thead>

<tbody>
REPLACE_DATA
</tbody>

</table>
</div>

'''

In [63]:
qual_triplets_vis = {}
for sample_id, triplet_set in triplet_data.items():
    xml_set = ''
    for this_set in triplet_set:
        for item in this_set:
            subj, pred, obj = item
            this_xml_triplet = html_triplet.replace('SUBJ', subj).replace('PRED', pred).replace('OBJ', obj)
            xml_set += this_xml_triplet
    full_xml_table = html_head.replace('REPLACE_DATA', xml_set)
    qual_triplets_vis[sample_id] = full_xml_table

In [66]:
len(qual_triplets_vis)

178

In [65]:
with open('../inputs/triples_REF.json', 'w') as f0:
    json.dump(qual_triplets_vis, f0)

In [69]:
# prepare texts from the baseline and submissions

submission_path = '../../../submissions2020/RDF-to-Text(English)/sub/'

with open(submission_path + 'baseline_WebNLG-2020-FORGe2020-12-2_submission.en', 'r') as f1:
    texts_BASELINE = f1.read().splitlines()


In [78]:
texts = {}
for num, text in enumerate(texts_BASELINE, 1):
    if str(num) in sampled_ids:
        texts[num] = text

In [79]:
with open('./texts_BASELINE.json', 'w') as f4:
    json.dump(texts, f4)

In [119]:
with open('./subm_paths.json', 'r') as f6:
    subm_dict = json.load(f6)

In [117]:
for k, v in subm_dict.items():
    
    with open(submission_path + v, 'r') as f:
        texts = f.read().splitlines()
    sampled_texts = {}
    
    for num, text in enumerate(texts, 1):
        if str(num) in sampled_ids:
            sampled_texts[num] = text
            
    with open(f'./texts_{k}.json', 'w') as f4:
        json.dump(sampled_texts, f4)